In [1]:
# Text Relevant Comparation Example's Book
import pandas as pd
from openai import OpenAI
import boto3, json
import numpy as np

AWS_ACCESS_KEY_ID="XXXXX"
AWS_SECRET_ACCESS_KEY="XXXXX"
AWS_SESSION_TOKEN="XXXXX"

bedrock_runtime = boto3.client(
    'bedrock-runtime', 
    region_name="us-east-1",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    )

# Note :To setup openai on your local , https://pypi.org/project/openai/
client = OpenAI()

In [2]:
# Create a document / Context
document_context = '''
Enter the context / document here.
'''

# Create Prompt
user_prompt ='''
Enter a prompt asking about the context / document here.
'''

In [3]:
# Embedding service (COHERE) : Change normal texts into numerical data
# Text can send with array format , has at least 1 element/array
 
def get_relative_embeddings(bedrock_runtime , body_array:[]):
    accept_message = "application/json"
    content_type_message = "application/json"
    model = "cohere.embed-multilingual-v3"
    body_json = {
        "texts": body_array,
        "input_type":"search_document",
    }
    body_message = json.dumps(body_json, indent=2, ensure_ascii=False)
    try:
        response = bedrock_runtime.invoke_model(body=body_message,
                                             contentType=content_type_message,
                                             accept=accept_message,
                                             modelId=model)
        response_body = json.loads(response["body"].read())
        embeddings = response_body['embeddings']
        return embeddings
    except Exception as e:
        raise e


In [4]:
# Calculation function : Find how close of context between Text(a) and Text(b)

def calculate_similarity(a, b):
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [5]:
# LLM function : Call Openai GPT 3.5 Turbo for generating text answer

def get_openai_answer(document_context,prompt):
    system_prompt = f'You are a helpful assistant.Please provide a concise summary of the following context: {document_context}'
    max_n_token = 1024
    completion_config = {
        "temperature": 1,
        "max_tokens": max_n_token,
    }

    try:
        response = client.chat.completions.create(model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": system_prompt},
            {'role': 'user', 'content': prompt},
        ],
        **completion_config)
        openai_response_text = response.choices[0].message.content.strip()
        return openai_response_text
    
    except Exception as e:
        raise e
    


In [6]:
# LLM function : Call Claude V2 for generating text answer

def get_claude_v2_answer(document_context, prompt):
    prompt = f'You are a helpful assistant.Please provide a concise summary ,based on given question, of the following context:{document_context}. Question:{prompt}'
    accept_message = "application/json"
    content_type_message = "application/json"
    model_message = "anthropic.claude-v2"
    body_message = json.dumps(
        {
            "prompt": "\n\nHuman:"+ prompt +"\n\nAssistant:",
            "max_tokens_to_sample": 1024,
        }
    ).encode()

    try:
        bedrock_response = bedrock_runtime.invoke_model(body=body_message,
                                            contentType=content_type_message,
                                            accept=accept_message,
                                            modelId=model_message)
        claude_response_text = json.loads(bedrock_response["body"].read())

        return claude_response_text['completion']
    except Exception as e:
        raise e

In [7]:
# LLM function : Call Claude V3 for generating text answer

def get_claude_v3_answer(document_context, prompt):
    prompt = f'You are a helpful assistant.Please provide a concise summary ,based on given question, of the following context:{document_context}. Question:{prompt}'
    accept_message = "application/json"
    content_type_message = "application/json"
    model_message = "anthropic.claude-3-haiku-20240307-v1:0"
    messages_api_body = json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 1000,
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": prompt
                            }
                        ]
                    }
                ]
            }
    )

    try:
        bedrock_response = bedrock_runtime.invoke_model(body=messages_api_body,
                                            contentType=content_type_message,
                                            accept=accept_message,
                                            modelId=model_message)
        claude_v3_response_body = json.loads(bedrock_response["body"].read())
        return claude_v3_response_body["content"][0]["text"]
    
    except Exception as e:
        raise e


In [8]:
# Start getting result from 3 LLMs
openai_response_text = get_openai_answer(document_context=document_context,prompt=user_prompt)
claude_v2_response_text = get_claude_v2_answer(document_context=document_context, prompt=user_prompt)
claude_v3_response_text = get_claude_v3_answer(document_context=document_context, prompt=user_prompt)

In [9]:
# Send Original Context to get embeddings
text_original = [document_context]
original_embeddings_result = get_relative_embeddings(bedrock_runtime=bedrock_runtime, body_array=text_original)

# Send Openai Completion to get embeddings
text_openai = [openai_response_text]
openai_embeddings_result = get_relative_embeddings(bedrock_runtime=bedrock_runtime, body_array=text_openai)

# Send ClaudeV2, V3 Completion to get embeddings
text_pair_claude = [claude_v2_response_text,claude_v3_response_text]
claude_embeddings_result = get_relative_embeddings(bedrock_runtime=bedrock_runtime, body_array=text_pair_claude)

In [10]:
# List1
name = ['GPT-3.5-Turbo', 'Claude-2','Claude-3-Haiku',]
# List2
cosine_socre_openai = calculate_similarity(original_embeddings_result[0],openai_embeddings_result[0])
cosine_score_claudeV2 = calculate_similarity(original_embeddings_result[0],claude_embeddings_result[0])
cosine_score_claudeV3 = calculate_similarity(original_embeddings_result[0],claude_embeddings_result[1])
score = [cosine_socre_openai, cosine_score_claudeV2,cosine_score_claudeV3,]

# get the list of tuples from two lists.
list_of_tuples = list(zip(name, score))

# Converting lists of tuples into
# pandas Dataframe.
df = pd.DataFrame(list_of_tuples,
				columns=['Name', 'Score'])

# Print score data.
print('-' * 50)
print(df)
print('-' * 50)


--------------------------------------------------
             Name     Score
0   GPT-3.5-Turbo  0.896029
1        Claude-2  0.867688
2  Claude-3-Haiku  0.896028
--------------------------------------------------


<!-- Citation -->